In [1]:
from unicorn_binance_websocket_api.unicorn_binance_websocket_api_manager import BinanceWebSocketApiManager
import os
import requests
import sys
import time
import threading
import logging


logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

try:
    from binance.client import Client
except ImportError:
    print("Please install `python-binance`! https://pypi.org/project/python-binance/#description")
    sys.exit(1)


def print_stream_data_from_stream_buffer(binance_websocket_api_manager):
    while True:
        if binance_websocket_api_manager.is_manager_stopping():
            exit(0)
        oldest_stream_data_from_stream_buffer = binance_websocket_api_manager.pop_stream_data_from_stream_buffer()
        if oldest_stream_data_from_stream_buffer is not False:
            pass
        else:
            time.sleep(0.01)


binance_api_key = ""
binance_api_secret = ""

#channels = {'aggTrade', 'trade', 'kline_1m', 'kline_5m', 'kline_15m', 'kline_30m', 'kline_1h', 'kline_2h', 'kline_4h',
#            'kline_6h', 'kline_8h', 'kline_12h', 'kline_1d', 'kline_3d', 'kline_1w', 'kline_1M', 'miniTicker',
#            'ticker', 'bookTicker', 'depth5', 'depth10', 'depth20', 'depth'}
#channels.add('depth@100ms')
channels = {'aggTrade', 'trade', 'kline_1m'}
arr_channels = {'!miniTicker', '!ticker', '!bookTicker'}
markets = []

try:
    binance_rest_client = Client(binance_api_key, binance_api_secret)
except requests.exceptions.ConnectionError:
    print("No internet connection?")
    sys.exit(1)

ubwa = BinanceWebSocketApiManager()

# start a worker process to move the received stream_data from the stream_buffer to a print function
worker_thread = threading.Thread(target=print_stream_data_from_stream_buffer, args=(ubwa,))
worker_thread.start()

data = binance_rest_client.get_all_tickers()
for item in data:
    markets.append(item['symbol'])

stream_id_arr = ubwa.create_stream(arr_channels, "arr", stream_label="arr channels")

max_subscriptions = ubwa.get_limit_of_subscriptions_per_stream()

for channel in channels:
    if len(markets) <= max_subscriptions:
        ubwa.create_stream(channel, markets, stream_label=channel)
    else:
        loops = 1
        i = 1
        markets_sub = []
        for market in markets:
            markets_sub.append(market)
            if i == max_subscriptions or loops*max_subscriptions + i == len(markets):
                ubwa.create_stream(channel, markets_sub, stream_label=str(channel+"_"+str(i)))
                markets_sub = []
                i = 1
                loops += 1
            i += 1


In [2]:
ubwa.print_summary()


====================== unicorn-binance-websocket-api_1.24.0-python_3.6.9 ======================
 exchange: binance.com
 uptime: 30 seconds since 2020-10-08, 22:35:26 UTC
 streams: 7
 active_streams: 7
 subscriptions: 3267
 current_receiving_speed: 193.35 kB/s
 average_receiving_speed: 183.73 kB/s (per day 15.14 gB)
 highest_receiving_speed: 430.44 kB/s (reached at 2020-10-08, 22:35:30 UTC)
 total_receives: 4293
 total_received_bytes: 5774063 (5.51 mB)
 total_transmitted_payloads: 13
 process_ressource_usage: cpu=30.8%, memory=76.86 mB, threads=22
 ---------------------------------------------------------------------------------------------
               stream_id              |   stream_label  |  last  |  average  |  most  | recon
 ---------------------------------------------------------------------------------------------
 fb17bedb-1f95-42f9-954f-36762e3a7689 | arr channels    |      2 |      1.92 |      3 |      0 
 31f9f15d-a38f-409f-8fda-46910742db2f | aggTrade_65     |      7 | 

In [3]:
ubwa.get_errors_from_endpoints()

[]

In [4]:
ubwa.get_results_from_endpoints()

['{"result":null,"id":6}',
 '{"result":null,"id":1}',
 '{"result":null,"id":10}',
 '{"result":null,"id":2}',
 '{"result":null,"id":7}',
 '{"result":null,"id":8}',
 '{"result":null,"id":9}',
 '{"result":null,"id":3}',
 '{"result":null,"id":4}',
 '{"result":null,"id":5}',
 '{"result":null,"id":11}',
 '{"result":null,"id":12}',
 '{"result":null,"id":13}']

In [5]:
ubwa.get_monitoring_status_plain()

{'active_streams': 7,
 'average_receives_per_second': 189.31,
 'average_speed_per_second': 208.46,
 'crashed_streams': 0,
 'highest_restart_per_stream_last_hour': 0,
 'reconnects': 0,
 'restarting_streams': 0,
 'return_code': 1,
 'status_msg': '',
 'status_text': 'WARNING',
 'stopped_streams': 0,
 'stream_buffer_items': '1',
 'stream_buffer_mb': 0.0011,
 'timestamp': 1602196573.038726,
 'total_received_length': 8805,
 'total_received_mb': 9.47,
 'update_msg': ' Update available: UnicornFy and check_lucit_collector.py!',
 'uptime': 0.001}

In [6]:
ubwa.get_number_of_free_subscription_slots(stream_id_arr)

1021

In [6]:
ubwa.get_number_of_subscriptions(stream_id_arr)

3

In [7]:
ubwa.print_stream_info(stream_id_arr)

====================== unicorn-binance-websocket-api_1.24.0-python_3.6.9 ======================
 exchange: binance.com 
 stream_id: fb17bedb-1f95-42f9-954f-36762e3a7689 
 stream_label: arr channels
 channels (3): ['!miniTicker', '!bookTicker', '!ticker'] 
 markets (1): ['arr'] 
 subscriptions: 3
 status: running
 start_time: 1602196526.950256 
 uptime: 55 seconds since 2020-10-08, 22:35:26 UTC
 reconnects: 0  
 last_heartbeat: 1602196581.7888386 
 seconds_to_last_heartbeat: 0.1909334659576416 
 kill_request: None 
 stop_request: None 
 has_stopped: False 
 seconds_since_has_stopped: None 
 current_receiving_speed: 279.0 kB/s 
 processed_receives: 108 
 transmitted_payloads: 1 
 stream_most_receives_per_second: 3 
 stream_receives_per_second: 1.963 
 stream_receives_per_minute: 0 
 stream_receives_per_hour: 0 
 stream_receives_per_day: 0 



In [8]:
ubwa.get_stream_subscriptions(stream_id_arr)

True

In [9]:
ubwa.get_results_from_endpoints()

['{"result":null,"id":6}',
 '{"result":null,"id":1}',
 '{"result":null,"id":10}',
 '{"result":null,"id":2}',
 '{"result":null,"id":7}',
 '{"result":null,"id":8}',
 '{"result":null,"id":9}',
 '{"result":null,"id":3}',
 '{"result":null,"id":4}',
 '{"result":null,"id":5}',
 '{"result":null,"id":11}',
 '{"result":null,"id":12}',
 '{"result":null,"id":13}',
 '{"result":["!ticker@arr","!bookTicker@arr","!miniTicker@arr"],"id":14}']

In [10]:
ubwa.unsubscribe_from_stream(stream_id_arr, channels="!ticker")


True

In [11]:
ubwa.print_stream_info(stream_id_arr)

====================== unicorn-binance-websocket-api_1.24.0-python_3.6.9 ======================
 exchange: binance.com 
 stream_id: fb17bedb-1f95-42f9-954f-36762e3a7689 
 stream_label: arr channels
 channels (2): ['!miniTicker', '!bookTicker'] 
 markets (1): ['arr'] 
 subscriptions: 2
 status: running
 start_time: 1602196526.950256 
 uptime: 55 seconds since 2020-10-08, 22:35:26 UTC
 reconnects: 0  
 last_heartbeat: 1602196594.463804 
 seconds_to_last_heartbeat: 0.10397982597351074 
 kill_request: None 
 stop_request: None 
 has_stopped: False 
 seconds_since_has_stopped: None 
 current_receiving_speed: 125.81 kB/s 
 processed_receives: 135 
 transmitted_payloads: 3 
 stream_most_receives_per_second: 4 
 stream_receives_per_second: 1.963 
 stream_receives_per_minute: 0 
 stream_receives_per_hour: 0 
 stream_receives_per_day: 0 



In [12]:
big_kline_1m_stream_id = ubwa.get_stream_id_by_label("kline_1m_1024")

In [13]:
ubwa.print_stream_info(big_kline_1m_stream_id)

====================== unicorn-binance-websocket-api_1.24.0-python_3.6.9 ======================
 exchange: binance.com 
 stream_id: 756e082f-b451-4319-b1d2-0f672087bc80 
 stream_label: kline_1m_1024
 channels (1): ['kline_1m'] 


 markets (1023): ['bchbtc', 'fiobnb', 'docketh', 'arketh', 'lunbtc', 'tomousdt', 'gtobnb', 'bchsvpax', 'ongbtc', 'xmrusdt', 'blzusdt', 'trbusdt', 'renbtc', 'xembtc', 'yoyobnb', 'trbbtc', 'oaxbtc', 'blzbnb', 'btcusdc', 'bnbbusd', 'wtcusdt', 'rdnbnb', 'ltousdt', 'veneth', 'balbusd', 'wnxmusdt', 'cvcbtc', 'bullusdt', 'xtzbusd', 'phbbnb', 'btcdownusdt', 'xrpupusdt', 'lunabusd', 'gxsusdt', 'xrpgbp', 'linkupusdt', 'ctxcbtc', 'adaupusdt', 'linkaud', 'powreth', 'thetabnb', 'dltbtc', 'tfuelusdt', 'avabtc', 'ongusdt', 'onteth', 'keyusdt', 'xzcbtc', 'kavabnb', 'fuelbtc', 'paxbtc', 'stratbnb', 'databusd', 'gtousdt', 'bttusdt', 'irisbusd', 'xmrbusd', 'powrbtc', 'bnbusdc', 'ognbnb', 'btcgbp', 'snxbnb', 'evxbtc', 'sxpusdt', 'bnbdownusdt', 'srmbtc', 'dotbtc', 'ornusdt', 'ontbnb', 'linkusdt', 'rsrbusd', 'kmdbtc', 'perlusdt', 'ctxcusdt', 'btsbusd', 'arneth', 'poeeth', 'belbnb', 'wavesusdt', 'xrprub', 'enjbtc', 'zeneth', 'beambnb', 'diabtc', 'btceur', 'celrusdt', 'bakebnb', 'iostbnb', 'bnbupusdt', 'cmtet


 subscriptions: 1023
 status: running
 start_time: 1602196526.9676583 
 uptime: 2m:57s since 2020-10-08, 22:35:26 UTC
 reconnects: 0  
 last_heartbeat: 1602196704.5306242 
 seconds_to_last_heartbeat: 0.021552085876464844 
 kill_request: None 
 stop_request: None 
 has_stopped: False 
 seconds_since_has_stopped: None 
 current_receiving_speed: 6.47 kB/s 
 processed_receives: 7361 
 transmitted_payloads: 3 
 stream_most_receives_per_second: 719 
 stream_receives_per_second: 41.451 
 stream_receives_per_minute: 2487.042 
 stream_receives_per_hour: 0 
 stream_receives_per_day: 0 

